# Численное решение задачи Коши для системы обыкновенных дифференциальных уравнений
## Условие задачи (Вариант №7)
Для решения задачи Коши системы обыкновенных дифференциальных уравнений

$$ \begin{cases} 
\dfrac{dy_1}{dx} = -101y_1 + 250y_2 \\
\dfrac{dy_2}{dx} = 40y_1 - 101y_2 \\
y_1(0) = A \\
y_2(0) = B \quad\quad\quad \text{(здесь $А$, $B$, $D$ - константы)}\\ 
0 < x < D = 1\end{cases} $$

предложить численный метод, исследовать его на аппроксимацию и устойчивость, реализовать в виде программы, написанной на языке высокого уровня, произвести отладку и расчёты с относительной погрешностью $\delta = 10^{-6}$ к аналитическому решению задачи.

In [1]:
# Введите значения переменных A, B и D ниже:

A = 1.0
B = 1.0
D = 1.0

import numpy as np
coef_matrix = np.array([[-101, 250], [40, -101]])

## Аналитическое решение

1. Найдём собственные числа:

$$ \det\left(\begin{matrix}
-101 - \lambda & 250 \\
40 & - 101-\lambda
\end{matrix}\right) = (101+\lambda)^2 -250\cdot40 = \lambda^2 + 202\lambda+201 = (\lambda + 201)(\lambda + 1) = 0$$
$$ \lambda_1 = -1, ~\lambda_2 = -201$$


2. Найдём собственные вектора:

$$ \left(\begin{matrix}
-101 - \lambda_1 & 250 \\
40 & - 101-\lambda_1
\end{matrix}\right)
\left(\begin{matrix}
x \\
y
\end{matrix}\right) = 
\left(\begin{matrix}
0 \\
0
\end{matrix}\right)$$

$$ \left(\begin{matrix}
-100 & 250 \\
40 & - 100
\end{matrix}\right)
\left(\begin{matrix}
x \\
y
\end{matrix}\right) = 
\left(\begin{matrix}
0 \\
0
\end{matrix}\right)$$

$$\vec{v_1} = \left(\begin{matrix}
5 \\
2
\end{matrix}\right)$$


$$ \left(\begin{matrix}
-101 - \lambda_2 & 250 \\
40 & - 101-\lambda_2
\end{matrix}\right)
\left(\begin{matrix}
x \\
y
\end{matrix}\right) = 
\left(\begin{matrix}
0 \\
0
\end{matrix}\right)$$

$$ \left(\begin{matrix}
100 & 250 \\
40 & 100
\end{matrix}\right)
\left(\begin{matrix}
x \\
y
\end{matrix}\right) = 
\left(\begin{matrix}
0 \\
0
\end{matrix}\right)$$

$$\vec{v_2} = \left(\begin{matrix}
5 \\
-2
\end{matrix}\right)$$


3. Тогда общее решение системы дифференциальных уравнений будет иметь вид:

$$\vec{y}(x) = C_1 \vec{v_1} e^{-x} + C_2 \vec{v_2} e^{-201x}, \quad C_1,C_2 \in \mathbb{R}$$

$$\vec{y}(x) = C_1 \left(\begin{matrix}
5 \\
2
\end{matrix}\right) e^{-x} + C_2 \left(\begin{matrix}
5 \\
-2
\end{matrix}\right) e^{-201x}$$


4. Подставляя общее решение в начальные условия находим $C_1$ и $C_2$:

$$\begin{cases}
y_1(0) = 5C_1\cdot e^{-0} + 5C_2 \cdot e^{-201\cdot0} = 5(C_1 + C_2) = A \\
y_2(0) = 2C_1\cdot e^{-0} - 2C_2 \cdot e^{-201\cdot0} = 2(C_1 - C_2) = B 
\end{cases}$$

In [2]:
# Решение системы линейных уравнений - поиск коэффициентов C_1 и C_2

import math
import numbers

analyt_coef_matrix = np.array([[5, 5], [2, -2]])
analyt_ord_val_matrix = np.array([A, B])

c_vec = np.linalg.solve(analyt_coef_matrix, analyt_ord_val_matrix)

if np.allclose(np.dot(analyt_coef_matrix, c_vec), analyt_ord_val_matrix) == False:
    print u"Вырожденная матрица, решение не существует"
else:
    print "C_1 = ", c_vec[0]
    print "C_2 = ", c_vec[1]

C_1 =  0.35
C_2 =  -0.15


In [3]:
# Аналитическое решение системы

def AnalyticalSolution(x):
    
    if isinstance(x, numbers.Number):
        ValueError("Argument x must be a number")
    
    v_1 = np.array([[5], [2]])  #
    v_2 = np.array([[5], [-2]])
    
    sol_vec = c_vec[0] * math.exp(-x) * v_1 + c_vec[1] * math.exp(-201 * x) * v_2
 
    return sol_vec

#print ComputeAnalyticalSolution(0.5)

## Численное решение

### Метод

Воспользуемся явным методом Эйлера:

$$ \vec{y}_{n+1} = \vec{y}_n + h f_n = \vec{y}_n + h \cdot A\vec{y}_n $$

где $h$ - шаг разностной сетки, а 
$$A = \left(\begin{matrix}
-101 & 250 \\
40 & -101
\end{matrix}\right)$$

Этот метод является методом Рунге-Кутты первого порядка аппроксимации и задаётся таблицей Бутчера

$$\left[\begin{matrix}
0 & 0 \\
0 & 1
\end{matrix}\right]$$


### Аппроксимация

Из формулы метода получаем

$$ \vec{y}_{n+1} = (E + hA)\vec{y}_n $$

$$ \vec{y}_{n} = (E + hA)^n \vec{y}_0 $$

Запишем теперь невязку метода

$$ \vec{y}_{n+1} - \vec{y}_{n} = hA\vec{y}_n  = hA(E + hA)^n \vec{y}_0 $$

Обозначая $C = ||A(E + 2 \cdot A)^n \vec{y}_0||$ (h < 2), то, с учётом того, что $h < 2$, т. к. длина отрезка интегрирования меньше единицы, получаем, что

$$ ||\vec{y}_{n+1} - \vec{y}_{n}|| = h \cdot ||A(E + hA)^n ~\vec{y}_0|| \leqslant h \cdot C$$

Отсюда по определению следует, что метод имеет первый порядок аппроксимации.


### Устойчивость

Представим задачу $$ \dfrac{d\vec{y}}{dt} = \vec{f}(x, \vec{u}) $$
в дискретном виде $$ \dfrac{\vec{y_{n+1}} - \vec{y_n}}{h} = \vec{R}(x, \vec{y_n})$$
где $\vec{R}(x,\vec{y})$ - функция приращения метода Рунге-Кутты. Для явного метода Эйлера имеем

$$\dfrac{\vec{\Delta}_{n+1}-\vec{\Delta}_{n}}{h} = J(x_n)\vec{\Delta}_{n+1}=(E+hJ(x_n))\vec{\Delta}_{n}$$

где $\vec{\Delta} = \vec{y}(x) - \vec{\varphi}(x)$ ($\varphi(x)$ - решение ОДУ).

$$ \vec{\Delta}_{n+1} = R(Z)\vec{\Delta}_{n} $$

$$ R(z) = 1 + z , \quad z = h\lambda$$

$$ |R(z)| = |1+z| = |1+x+iy| \leqslant 1$$

Для нашей задачи

$$ \vec{\Delta}_{n+1} = R^n(z)\vec{\Delta}_{n} = (1+\lambda_{1,2}h)^{n+1}\vec{\Delta}_{n+1}$$

$$ \begin{cases}
|1-201h| \leqslant 1 \\
|1-h| \leqslant 1 \\
\end{cases}$$

$$0 \leqslant h \leqslant \dfrac{2}{201} \approx 0.00995$$

Итак, метод будет устойчив, если выполнено ограничение на шаг сетки $g < 9\cdot10^{-3}$.

## Другие методы численного решения

Представленная задача является жёсткой. Действительно, 

$$\dfrac{\max_i |Re(\lambda_i)|}{\min_i |Re(\lambda_i)|} = \dfrac{201}{1} >> 1$$

Рассмотрим теперь несколько неявных методов Рунге-Кутта, которые должны быть более эффективны для решения жёстких задач и сравним их с явным методом Эйлера.

### Неявный метод Эйлера

$$ \dfrac{\vec{y}_{n+1} + \vec{y}_n}{h} = \vec{f}\left(\vec{y}_{n+1}\right) = A\vec{y_n}$$

$$ \vec{y}_{n+1} = (E - hA)^{-1} \vec{y}_n$$

Из теории известно, что данный метод имеет первый порядок аппроксимации и является устойчивым везде, кроме единичного круга с центром в точке $(1,0)$. Оба собственные числа отрицательные, поэтому для данной задачи метод устойчивый.

### Неявный метод трапеций

$$ \dfrac{\vec{y}_{n+1} + \vec{y}_n}{h} = \dfrac{\vec{f}\left(\vec{y}_{n+1}\right) + \vec{f}\left(\vec{y}_{n}\right) }{2} = \dfrac{A\vec{y}_{n+1}+A\vec{y}_n}{2}$$

$$ \vec{y}_{n+1} = \left(E - \frac{h}{2}A\right)^{-1}\left(E + \frac{h}{2}A\right) \vec{y}_n$$

Из теории известно, что данный метод  имеет второй порядок аппроксимации и является устойчивым в левой полуплоскости. Оба собственные числа отрицательные, поэтому для данной задачи метод устойчивый.

In [4]:
# Явный метод решения ОДУ в общем виде.

from numpy.linalg import inv

res_table = [["x", "y_1*", "y_1", "|y_1*-y_1|", "d_1", "y_2*", "y_2", "|y_2*-y_2|", "d_2"]]
col_num = len(res_table[0])

coef_matrix = np.array([[-101, 250], 
                        [40, -101]])

s = 1
rg_coefs = np.zeros((s + 1, 2, 1))  # массив с коэффициентами Рунге-Кутта
butch_matrix = np.array([[0, 0], 
                         [0, 1]])   # таблица Бутчера


def ComputeRGCoefficients(sol_u_n):
    for i in range (0, s + 1):

        rg_sum = np.zeros((2, 1))

        for j in range (1, i - 1):
            rg_sum += butch_matrix[i][j] * rg_coefs[j]

        rg_coefs[i] = np.matmul(coef_matrix, (sol_u_n + INT_STEP * rg_sum))
    
    return


def NumericalSolution(x_limit, int_step):
    
    if isinstance(x_limit, numbers.Number):
        ValueError("Argument x must be a number")

    if x_limit <= 0 or x_limit > D:
        ValueError("Argument x is out of integration range")

    y_n   = np.zeros((2, 1)) # y_n - solution on the n-th iteration
    y_n_1 = np.zeros((2, 1)) # y_n_1 = y_{n+1} - solution on the n+1-th iteration

    y_n[0] = A
    y_n[1] = B
    y_n_1 = y_n

    x = 0.0
    iter_num = 0
    
    while x < x_limit:
        
        sol_analytical = AnalyticalSolution(x)
                
        res_table.append([x, 
                          round(y_n[0], 7), 
                          round(sol_analytical[0][0], 7),
                          round(abs(y_n[0] - sol_analytical[0][0]), 7),
                          round(abs(y_n[0] - sol_analytical[0][0]) / sol_analytical[0][0], 7),                         round(y_n[1], 7), 
                          round(sol_analytical[1][0], 7),
                          round(abs(y_n[1] - sol_analytical[1][0]), 7),
                          round(abs(y_n[1] - sol_analytical[1][0]) / sol_analytical[1][0], 7)])
        
        x += int_step
        iter_num += 1
                
        ComputeRGCoefficients(y_n)

        rg_sum = np.zeros((2, 1))

        for k in range (0, s + 1):
            rg_sum += butch_matrix[s][k] * rg_coefs[k]

        #y_n_1 = y_n + int_step * np.matmul(coef_matrix, y_n)
        
        #  Явный метод Рунге-Кутта - явный метод Эйлера сходится с заданной точностью при шаге 0.00001
        y_n_1 = y_n + int_step * rg_sum
        
        #  Неявный метод Эйлера - расходится сходится с заданной точностью при шаге 0.00001
        #y_n_1 = np.matmul(inv(np.identity(2) - int_step * coef_matrix), y_n)
        
        #  Неявный метод трапеций - сходится с заданной точностью при шаге 0.01
        #y_n_1 = np.matmul(np.matmul(inv(np.identity(2) - int_step / 2 * coef_matrix), 
        #                                np.identity(2) + int_step / 2 * coef_matrix), y_n) 
        
        y_n = y_n_1
        
        #print "x = ", x, " u_x = ", y_n[0]

    return y_n_1


INT_STEP = 0.00001

NumericalSolution(D, INT_STEP)


array([[0.64377937],
       [0.25751175]])

In [5]:
# Сравнение аналитического и численного решения - вывод таблицы

header_format = "{:<7}  " + "{:<11}  " * (col_num - 1)
row_format    = "{:.1f} " + " " * 5 + "{:<11}  " * (col_num - 1) 
# docs.python.org/3/library/string.html#formatspec

POINTS_2_PRINT = 10
tab_step = int(round((D / INT_STEP / POINTS_2_PRINT)))
cur_line  = 0
cur_point = 0

print header_format.format(*res_table[0])


while cur_point < POINTS_2_PRINT:

    cur_line += tab_step
    
    print row_format.format(*res_table[cur_line])
        
    cur_point += 1
    
#print res_table

x        y_1*         y_1          |y_1*-y_1|   d_1          y_2*         y_2          |y_2*-y_2|   d_2          
0.1      1.5834805    1.5834813    8e-07        5e-07        0.6333922    0.6333925    3e-07        5e-07        
0.2      1.4327917    1.4327931    1.4e-06      1e-06        0.5731167    0.5731173    6e-07        1e-06        
0.3      1.2964429    1.2964449    1.9e-06      1.5e-06      0.5185772    0.5185779    8e-07        1.5e-06      
0.4      1.1730695    1.1730718    2.3e-06      2e-06        0.4692278    0.4692287    9e-07        2e-06        
0.5      1.0614366    1.0614393    2.7e-06      2.5e-06      0.4245746    0.4245757    1.1e-06      2.5e-06      
0.6      0.9604271    0.96043      2.9e-06      3e-06        0.3841708    0.384172     1.2e-06      3e-06        
0.7      0.8690299    0.869033     3e-06        3.5e-06      0.347612     0.3476132    1.2e-06      3.5e-06      
0.8      0.7863304    0.7863336    3.1e-06      4e-06        0.3145322    0.3145334    1

In [6]:
# Построение графика решения

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig = plt.figure(figsize = (8, 12))
fig.suptitle(u'График зависимости численного и аналитического решения задачи Коши от координаты', fontsize = 16)

graph_1 = plt.subplot(211)
graph_1.set_title(u'По первой координате', fontsize = 14)
plt.xlabel(r'$x$'  , fontsize = 14, fontweight = 'bold')
plt.ylabel(r'$y_1$', fontsize = 14, fontweight = 'bold')
#graph_1.set_aspect(0.75)

graph_2 = plt.subplot(212)
graph_2.set_title(u'По второй координате', fontsize = 14)
plt.xlabel(r'$x$'  , fontsize = 14, fontweight = 'bold')
plt.ylabel(r'$y_2$', fontsize = 14, fontweight = 'bold')


i = 1

while i < len(res_table):
    plt.subplot(211)
    
    plt.plot(res_table[i][0], res_table[i][1], color = 'red',   marker = 'o', markersize = 5)
    plt.plot(res_table[i][0], res_table[i][2], color = 'blue',  marker = 'o', markersize = 3)
    
    plt.subplot(212)     
    plt.plot(res_table[i][0], res_table[i][5], color = 'red',   marker = 'o', linewidth = 2, markersize = 5)
    plt.plot(res_table[i][0], res_table[i][6], color = 'blue',  marker = 'o', linewidth = 2, markersize = 3)

    i += 1250
    
plt.subplots_adjust(top = 0.92, bottom = 0.08, left = 0.10, right = 0.95, hspace = 0.25, wspace = 0.55)
red_dot_patch  = mpatches.Patch(color = 'red',  label = 'Numerical')
blue_dot_patch = mpatches.Patch(color = 'blue', label = 'Analytical')

plt.subplot(211)
plt.legend(handles = [red_dot_patch, blue_dot_patch])         
plt.subplot(212)
plt.legend(handles = [red_dot_patch, blue_dot_patch])         

plt.show()

C:\Users\babus\Anaconda2\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


## Результаты и выводы

В результате работы предложены и реализованы устойчивый численные методы для решения представленной задачи Коши системы обыкновенных дифференциальных уравнений - явный метод Эйлера, неявный метод Эйлера и неявный метод трапеций. Данный метод имеет первый порядок аппроксимации. Для достижения точности с относительной погрешностью $\delta = 10^{-6}$ был выбран шаг метода равный $h_1 = 10^{-5}$, $h_2 = 10^{-5}$ и $h_3 = 10^{-2}$ соответственно. Зависимость решения от координаты представлена графиком (см. выше).

Прокомментируем полученные результаты. Преимуществами явного метода Эйлера перед другими методами являются простота его реализации и простота его теоретического анализа. При этом для достижения заданной точности требуется выбор крайне малого шага сетки - $10^{-5}$. Для сравнения, неявный метод трапеций для достижения той же точности требует шаг всего $10^{-2}$, а неявный метод Эёлера - $10^{-5}$.